# I. Train and test datasets

In [ ]:
# Hyperparameters for dataset

D = 784 #dimension
#D = 3
k = 3 # num of 2d planes in dim D
n = 6*(10**3) # num of points in each plane
d = 2 # starting dimention of gaussians
#n = 10**3 # num of points in each plane
shift_class = 0
intercl_var = 0.1 #initially 0.1


# Hyperparameters for data loaders
#batch_size  = 16
split_ratio = 0.2

# Set manual seed for reproducibility
# torch.manual_seed(0)

In [ ]:
# adding path to the set generating package
import sys
sys.path.append('../') # have to go 1 level up

In [ ]:
import torch
import ricci_regularization


# Generate dataset
# via classes
torch.manual_seed(0) # reproducibility
my_dataset = ricci_regularization.SyntheticDataset(k=k,n=n,d=d,D=D,
                                    shift_class=shift_class, intercl_var=intercl_var)

train_dataset = my_dataset.create

m = len(train_dataset)
train_data, test_data = torch.utils.data.random_split(train_dataset, [int(m-m*split_ratio), int(m*split_ratio)])


# II. Fit dimension reduction models

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    return plt.cm.get_cmap(base_cmap, N)

## II.1. TSNE check

In [ ]:
# TNSE check on test set
synthetic_points = test_data[:][0].view(-1,D)

tsne   = TSNE(n_components=2, verbose=1, random_state=123)
z_test = tsne.fit_transform(synthetic_points.numpy())

In [ ]:
plt.rcParams.update({'font.size': 20}) # makes all fonts on the plot be 20
plt.scatter( z_test[:,0], z_test[:,1], c=test_data[:][1], alpha=0.5, cmap=discrete_cmap(k,'jet'),marker='o',edgecolors=None )
plt.title( "t-SNE projection of the Synthetic dataset")
plt.colorbar(ticks=range(k),orientation='vertical',shrink = 0.7)
plt.show()

In [ ]:
# TNSE check on train set
"""
synthetic_points = train_data[:][0].view(-1,28*28)

tsne   = TSNE(n_components=2, verbose=1, random_state=123)
z_train = tsne.fit_transform(synthetic_points.numpy())
"""

In [ ]:
"""
plt.scatter( z_train[:,0], z_train[:,1], c=train_data[:][1], alpha=0.5 )
plt.title( "TSNE projection of train data")
plt.show()
"""

### Plots using Seaborn

In [ ]:
"""
import pandas as pd 

# Format data
df = pd.DataFrame()
#df["y"] = labels.numpy()
df["y"] = test_data[:][1].numpy() #test_data[:][1] are labels
df["comp-1"] = z_test[:,0]
df["comp-2"] = z_test[:,1]

import seaborn as sns
import numpy as np # this module is useful to work with numerical arrays
sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 10),
                data=df).set(title="Synthetic dataset data T-SNE projection")
"""

## II.2. UMAP

In [ ]:
points = test_data[:][0]
labels = test_data[:][1]

In [ ]:
points.shape

In [ ]:
import umap

mapper = umap.UMAP().fit(points)


In [ ]:
encoded_points = mapper.embedding_

In [ ]:
plt.rcParams.update({'font.size': 20}) # makes all fonts on the plot be 20
plt.scatter( encoded_points[:,0], encoded_points[:,1], c=test_data[:][1], alpha=0.5, cmap=discrete_cmap(k,'jet'),marker='o',edgecolors=None )
plt.title( "UMAP embedding of the Synthetic dataset")
plt.colorbar(ticks=range(k),orientation='vertical',shrink = 0.7)
plt.show()

# III. 3D visualization

In [ ]:
# only for D=3
if D==3:
    points = test_data[:][0].squeeze()
    labels = test_data[:][1]
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    plt.rcParams.update({'font.size': 20})
    ax.scatter(points[:,0],
               points[:,1],
               points[:,2],
               c=labels, s=30, alpha = 0.5,cmap=plt.cm.get_cmap('jet', k))
    plt.title("Synthetic dataset in 3d")
    ax.view_init(azim=160, elev=55)
    plt.show()